<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<font color="#138D75">**NERO Winter School training**</font> <br>
**Copyright:** (c) 2025 EUMETSAT <br>
**License:** GPL-3.0-or-later <br>
**Authors:** Dominika Leskow-Czyżewska (EUMETSAT), based on <a href='https://github.com/eu-cdse/notebook-samples/blob/main/openeo/Batch_job.ipynb'>OpenEO Copernicus Data Space Ecosystem Sample notebooks</a>.

# Sentinel-2 full resolution imagery - Using openEO Batch Jobs To Run Large and Heavy Workflows


Most of the simple, basic openEO usage examples show synchronous execution of process graphs:
you submit a process graph with a HTTP request and receive the result as direct response of that same request. 
This is only feasible if the processing doesn’t take too long (a couple of minutes at most).


For the heavier work, covering large regions of interest, long time series, more intensive processing, etc, you have to use batch jobs.

This notebook shows how to programmatically create and interact with batch job using the openEO Python client library.

## Set up

Import `openeo` package and establish an authenticated connection to Copernicus Data Space Ecosystem openEO back-end.

In [1]:
import openeo
from os import chmod

/home/jovyan/nero-winter-school-2025/nero_winter_school/lib/python3.11/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [3]:
# Uncomment the line below the the cell below encounters an error
# chmod('/home/jovyan/.local/share/openeo-python-client/refresh-tokens.json', 0o600)

In [4]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

## Build data cube

Start with a simple data cube: small spatiotemporal slice of `SENTINEL2_L2A` data:

In [5]:
#  lat-lon geographical bounds of search area
W = 23.3
S = 37.8
E = 24.5
N = 38.5

start_date = "2024-08-07"
end_date = "2024-08-18"

run_name = "varnavas_example"
output_dir = './example_data/'

bands = ["B08", "B04", "B03", "B02", "B12"]

In [6]:
datacube = connection.load_collection(
    "SENTINEL2_L2A",
    bands=bands,
    temporal_extent=(start_date, end_date),
    spatial_extent={
        "west": W,
        "south": S,
        "east": E,
        "north": N,
        "crs": "EPSG:4326",
    },
    max_cloud_cover=100,
)

Set up output format to be GeoTIFF:

In [7]:
job = datacube.save_result(format="GTiff")

## Run as Batch Job

The easiest way to run our processing as a batch job is using the `execute_batch()` helper,
which takes care of creating a batch job, starting it, and keep polling its status until it's finished (or failed).

While not necessary, it is recommended to give your batch job a descriptive title so it’s easier to identify in your job listing.

In [8]:
job = job.execute_batch(title="Slice of S2 data")
print("Download complete!")

0:00:00 Job 'j-2502191742104f609ad6db65dac8d066': send 'start'
0:00:13 Job 'j-2502191742104f609ad6db65dac8d066': created (progress 0%)
0:00:18 Job 'j-2502191742104f609ad6db65dac8d066': created (progress 0%)
0:00:24 Job 'j-2502191742104f609ad6db65dac8d066': created (progress 0%)
0:00:32 Job 'j-2502191742104f609ad6db65dac8d066': created (progress 0%)
0:00:42 Job 'j-2502191742104f609ad6db65dac8d066': created (progress 0%)
0:00:54 Job 'j-2502191742104f609ad6db65dac8d066': running (progress N/A)
0:01:10 Job 'j-2502191742104f609ad6db65dac8d066': running (progress N/A)
0:01:29 Job 'j-2502191742104f609ad6db65dac8d066': running (progress N/A)
0:01:53 Job 'j-2502191742104f609ad6db65dac8d066': running (progress N/A)
0:02:23 Job 'j-2502191742104f609ad6db65dac8d066': running (progress N/A)
0:03:00 Job 'j-2502191742104f609ad6db65dac8d066': running (progress N/A)
0:03:47 Job 'j-2502191742104f609ad6db65dac8d066': running (progress N/A)
0:04:45 Job 'j-2502191742104f609ad6db65dac8d066': running (progres

If you need a bit more control over the lifetime of a batch job, 
you can do each step manually, e.g. 
- create a job with `job = cube.create_job()`
- start a job with `job.start_job()`
- wait until `job.status()` reaches `"finished"`


## Optional: Inspecting a Job

A batch job on a back-end is fully identified by its job id. Especially if a job fails, it might be useful to check the logs to understand why it was the case.

In case of the job we created above:

In [9]:
job.job_id

'j-2502031342484645a2d4283c99fcad56'

It's recommended to properly take note of the batch job id.
It allows you to “reconnect” to your job (using `connection.job(job_id)`) on the back-end, 
even if it was created at another time, by another script/notebook or even with another openEO client.


A batch job typically takes some time to finish, and you can check its status with the `status()` method.

In [7]:
job.status()

'finished'

Batch job logs can be fetched with `job.logs()`. If you prefer a graphical, web-based interactive environment to manage and monitor your batch jobs, feel free to switch to an openEO web editor like [openeo.dataspace.copernicus.eu](https://openeo.dataspace.copernicus.eu/) at any time.

In [23]:
job.logs()

[{'id': '[1738658869688, 620896]',
  'time': '2025-02-04T08:47:49.688Z',
  'level': 'info',
  'message': 'Starting batch job os.getpid()=76: start 2025-02-04 08:47:49.687827'},
 {'id': '[1738658870431, 595373]',
  'time': '2025-02-04T08:47:50.431Z',
  'level': 'info',
  'message': "Loaded config config_id='os_creodias_openeo_k8s_prod' from config_path='/opt/backend_config/backendconfig.py' (reason='lazy_load')"},
 {'id': '[1738658871004, 9322]',
  'time': '2025-02-04T08:47:51.004Z',
  'level': 'info',
  'message': 'Logging initialized @5423ms to org.sparkproject.jetty.util.log.Slf4jLog'},
 {'id': '[1738658883081, 463622]',
  'time': '2025-02-04T08:48:03.081Z',
  'level': 'info',
  'message': 'Job spec: {\n "process_graph": {\n  "apply1": {\n   "process_id": "apply",\n   "arguments": {\n    "process": {\n     "process_graph": {\n      "multiply1": {\n       "result": true,\n       "process_id": "multiply",\n       "arguments": {\n        "x": {\n         "from_node": "linearscalerange1"\n        },\n        "y": 2.5\n       }\n      },\n      "linearscalerange1": {\n       "process_id": "linear_scale_range",\n       "arguments": {\n        "inputMax": 3000,\n        "inputMin": 0,\n        "x": {\n         "from_parameter": "x"\n        },\n        "outputMin": 0,\n        "outputMax": 255\n       }\n      }\n     }\n    },\n    "data": {\n     "from_node": "loadcollection1"\n    }\n   }\n  },\n  "loadcollection1": {\n   "process_id": "load_collection",\n   "arguments": {\n    "temporal_extent": [\n     "2024-08-07",\n     "2024-08-17"\n    ],\n    "spatial_extent": {\n     "east": 23.994827,\n     "south": 38.017263,\n     "crs": "EPSG:4326",\n     "north": 38.267837,\n     "west": 23.806\n    },\n    "id": "SENTINEL2_L2A",\n    "bands": [\n     "B08",\n     "B04",\n     "B03"\n    ],\n    "properties": {\n     "eo:cloud_cover": {\n      "process_graph": {\n       "lte1": {\n        "result": true,\n        "process_id": "lte",\n        "arguments": {\n         "x": {\n          "from_parameter": "value"\n         },\n         "y": 100\n        }\n       }\n      }\n     }\n    }\n   }\n  },\n  "saveresult1": {\n   "result": true,\n   "process_id": "save_result",\n   "arguments": {\n    "data": {\n     "from_node": "apply1"\n    },\n    "format": "GTiff",\n    "options": {}\n   }\n  }\n },\n "job_options": {}\n}'},
 {'id': '[1738658884845, 980026]',
  'time': '2025-02-04T08:48:04.845Z',
  'level': 'warning',
  'message': 'No elastic_job_registry given to GeoPySparkBackendImplementation, creating one'},
 {'id': '[1738658884965, 465825]',
  'time': '2025-02-04T08:48:04.965Z',
  'level': 'info',
  'message': "Doing 'client_credentials' token request 'https://sso.terrascope.be/auth/realms/terrascope/protocol/openid-connect/token' with post data fields ['grant_type', 'client_id', 'client_secret', 'scope'] (client_id 'openeo-elastic-job-registry')"},
 {'id': '[1738658885235, 360116]',
  'time': '2025-02-04T08:48:05.235Z',
  'level': 'info',
  'message': "EJR health check {'status': 'ok', 'info': {'elasticsearch-ping': {'status': 'up'}, 'elasticsearch-health': {'status': 'up'}, 'auth': {'status': 'up', 'state': 'ok'}}, 'error': {}, 'details': {}}"},
 {'id': '[1738658885881, 19107]',
  'time': '2025-02-04T08:48:05.881Z',
  'level': 'info',
  'message': 'Doing dry run'},
 {'id': '[1738658885881, 101295]',
  'time': '2025-02-04T08:48:05.881Z',
  'level': 'info',
  'message': 'Correlation id: j-2502040847274ffb8703cb9de0e1a379'},
 {'id': '[1738658885881, 851630]',
  'time': '2025-02-04T08:48:05.881Z',
  'level': 'info',
  'message': "Loaded config config_id='os_creodias_openeo_k8s_prod' from config_path='/opt/backend_config/backendconfig.py' (reason='lazy_load')"},
 {'id': '[1738658885882, 883762]',
  'time': '2025-02-04T08:48:05.882Z',
  'level': 'info',
  'message': "Dry run extracted these source constraints: [(('load_collection', ('SENTINEL2_L2A', (('eo:cloud_cover', (('lte', 100),)), ('productType', (('eq', 'L2A'),))), ('B08', 'B04'

## Fetch Batch Job Results

The result of a finished batch job consists of several elements:
- a STAC-compatible description (metadata) of the batch job results
- one or more output files (e.g. multiple GeoTIFF or netCDF assets)

You can get a handle to these results with  `get_results()`:

In [16]:
results = job.get_results()

In the general case, when you have one or more result files (also called “assets”), the easiest option to download them is using `download_files()` (plural) where you just specify a download folder (otherwise the current working directory will be used by default).

In [17]:
output_dir = os.path.join(output_dir, run_name, 'Satellite_Imagery', 'S2_DAILY')
results.download_files(output_dir)

[PosixPath('test/testrun/Satellite_Imagery/S2_DAILY/openEO_2024-08-07Z.tif'),
 PosixPath('test/testrun/Satellite_Imagery/S2_DAILY/openEO_2024-08-12Z.tif'),
 PosixPath('test/testrun/Satellite_Imagery/S2_DAILY/openEO_2024-08-17Z.tif'),
 PosixPath('test/testrun/Satellite_Imagery/S2_DAILY/job-results.json')]

<hr>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE.TXT">GPL-3.0-or-later</a> license <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/atmosphere/trainings/nero-winter-school-2025">View on GitLab</a> | <a href="https://classroom.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact</a></span></p>